In [2]:
using Revise

In [3]:
using ModelVerification

In [4]:
using LazySets
using PyCall
using CSV
using ONNX
using Flux
using ONNXNaiveNASflux
using Test
using NNlib
using NaiveNASflux

In [5]:
# using Flux: onehotbatch, onecold, flatten
# using Flux.Losses: logitcrossentropy
# using Statistics: mean
using CUDA
using MLDatasets: CIFAR10, MNIST
using MLUtils: splitobs, DataLoader
using Accessors
using Profile
using LinearAlgebra
using Einsum

In [32]:
function test_mlp(prop_method)
    small_nnet_file = "/home/verification/ModelVerification.jl/test/networks/small_nnet.nnet"
    # small_nnet encodes the simple function 24*max(x + 1.5, 0) + 18.5
    small_nnet = read_nnet(small_nnet_file, last_layer_activation = ModelVerification.ReLU())
    flux_model = Flux.Chain(small_nnet)
    #ONNXNaiveNASflux.save("/home/verification/ModelVerification.jl/small_nnet.onnx", flux_model, (1,1))
    #println(flux_model)
    #println(flux_model.layers[1].weight, " ", flux_model.layers[1].bias) # max(x+1.5, 0) max(x+1.5, 0)              [0,4]
    #println(flux_model.layers[2].weight, " ", flux_model.layers[2].bias) # 4*max(x+1.5, 0)+2.5 4*max(x+1.5, 0)+2.5  [2.5, 18.5]
    #println(flux_model.layers[3].weight, " ", flux_model.layers[3].bias) # 24*max(x+1.5, 0)+18.5                    [18.5, 114.5]
    in_hyper  = Hyperrectangle(low = [-2.5], high = [2.5]) # expected out: [18.5, 114.5]
    out_holds = Hyperrectangle(low = [18], high = [115.0]) # 
    out_violated    = Hyperrectangle(low = [19], high = [114]) #
    comp_holds    = Complement(Hyperrectangle(low = [115], high = [118])) # y ≤ 115 or 118 ≤ y
    comp_violated    = Complement(Hyperrectangle(low = [10], high = [19])) # y ≤ 10 or 19 ≤ y
    info = nothing
    search_method = BFS(max_iter=5, batch_size=1)
    split_method = Bisect(1)
    @test verify(search_method, split_method, prop_method, Problem("/home/verification/ModelVerification.jl/small_nnet.onnx", in_hyper, out_holds)).status == :holds
    @test verify(search_method, split_method, prop_method, Problem("/home/verification/ModelVerification.jl/small_nnet.onnx", in_hyper, out_violated)).status == :violated
    @test verify(search_method, split_method, prop_method, Problem("/home/verification/ModelVerification.jl/small_nnet.onnx", in_hyper, comp_holds)).status == :holds
    @test verify(search_method, split_method, prop_method, Problem("/home/verification/ModelVerification.jl/small_nnet.onnx", in_hyper, comp_violated)).status == :violated
    #= @test verify(search_method, split_method, prop_method, Problem(flux_model, in_hyper, out_holds)).status == :holds
    @test verify(search_method, split_method, prop_method, Problem(flux_model, in_hyper, out_violated)).status == :violated
    @test verify(search_method, split_method, prop_method, Problem(flux_model, in_hyper, comp_holds)).status == :holds
    @test verify(search_method, split_method, prop_method, Problem(flux_model, in_hyper, comp_violated)).status == :violated =#
end
@timed begin
    # #for i in 1:1
    #     test_mlp(Ai2z())
        # test_mlp(Crown(true, true))
    #     test_mlp(StarSet(Crown(true, true)))
        test_mlp(AlphaCrown(Crown(true, true), true, false, Flux.Optimiser(Flux.ADAM(0.1)), 10))
    #end
end

batch_bound.batch_data_min max
[-2.5;;][2.5;;]
---


1
[0.0;;]
---
2
[0.0; 0.0;;]
---
3
[8.333361500856284e-10; 8.333361500856284e-10;;]
batch_info[:init_lower_A_b]
Array{Float64}[[1.0; -1.0;;;], [-19.0; 10.0;;]]
out_l, out_u
[37.69999997999994;;;][37.70000008000027;;;]
spec_l, spec_u
[18.699999979999937; -85.30000000000001;;;][18.700000080000272; 10.700000000000003;;;]
Test Failed at /home/tianhao/dev/ModelVerification.jl/tmp/test.ipynb:22
  Expression: (verify(search_method, split_method, prop_method, Problem("/home/verification/ModelVerification.jl/small_nnet.onnx", in_hyper, comp_violated))).status == :violated
   Evaluated: holds == violated


Test.FallbackTestSetException: Test.FallbackTestSetException("There was an error during testing")

In [ ]:
model = Chain([
    Conv((3, 3), 3 => 8, relu, pad=SamePad(), stride=(2, 2)), #pad=SamePad() ensures size(output,d) == size(x,d) / stride.
    BatchNorm(8),
    MeanPool((2,2)),
    SkipConnection(
        Chain([
            Conv((5, 5), 8 => 8, relu, pad=SamePad(), stride=(1, 1))
            ]),
        +
    ),
    #ConvTranspose((3, 3), 8 => 4, relu, pad=SamePad(), stride=(2, 2)),#pad=SamePad() ensures size(output,d) == size(x,d) * stride.
    Flux.flatten,
    Dense(512, 100, relu),
    Dense(100, 10)
])
testmode!(model)
# image_seeds = CIFAR10(:train)[1:5].features # 32 x 32 x 3 x 5
image_seeds = [CIFAR10(:train)[i].features for i in 1:2]
input_set = ImageConvexHull(image_seeds)
# println(typeof(image_seeds[1][1,1,1,1]))
search_method = BFS(max_iter=1, batch_size=1)
split_method = Bisect(1)
output_set = BallInf(zeros(10), 1.0)
onnx_model_path = "/home/verification/ModelVerification.jl/mlp.onnx"
flux_model = model
image_shape = (32, 32, 3, 5)
println(image_seeds)

In [23]:
prop_method = ImageStar()
@timed verify(search_method, split_method, prop_method, Problem(flux_model, input_set, output_set))

In [ ]:
prop_method = ImageStarZono()
@timed verify(search_method, split_method, prop_method, Problem(onnx_model_path, image_seeds, output_set))

In [ ]:
model = Chain([
    Flux.flatten,
    Dense(784, 200, relu),
    Dense(200, 10)
])
image_seeds = [MNIST(:train)[i].features for i in 1:1]
search_method = BFS(max_iter=1, batch_size=1)
split_method = Bisect(1)
output_set = BallInf(zeros(10), 1.0)
onnx_model_path = "/home/verification/ModelVerification.jl/debug.onnx"
Flux_model = model
image_shape = (28, 28, 1, 1)

In [ ]:
model = Chain([
    Conv((3, 3), 3 => 8, relu, pad=SamePad(), stride=(2, 2)), #pad=SamePad() ensures size(output,d) == size(x,d) / stride.
    BatchNorm(8),
    MeanPool((2,2)),
    SkipConnection(
        Chain([
            Conv((5, 5), 8 => 8, relu, pad=SamePad(), stride=(1, 1))
            ]),
        +
    ),
    Conv((3, 3), 8 => 8, relu, pad=SamePad(), stride=(2, 2)),
    #ConvTranspose((3, 3), 8 => 4, relu, pad=SamePad(), stride=(2, 2)),#pad=SamePad() ensures size(output,d) == size(x,d) * stride.
    Flux.flatten,
    Dense(128, 100, relu),
    Dense(100, 10)
])
testmode!(model)
# image_seeds = CIFAR10(:train)[1:5].features # 32 x 32 x 3 x 5
image_seeds = [CIFAR10(:train)[i].features for i in 1:2]
# println(typeof(image_seeds[1][1,1,1,1]))
search_method = BFS(max_iter=1, batch_size=1)
split_method = Bisect(1)
output_set = BallInf(zeros(10), 1.0)
onnx_model_path = "/home/verification/ModelVerification.jl/mlp.onnx"
Flux_model = model
image_shape = (32, 32, 3, 2)

In [ ]:
prop_method = ImageStar()
@timed verify(search_method, split_method, prop_method, Problem(onnx_model_path, Flux_model, image_shape, image_seeds, output_set))

In [ ]:
prop_method = ImageStarZono()
@timed verify(search_method, split_method, prop_method, Problem(onnx_model_path, Flux_model, image_shape, image_seeds, output_set))

In [ ]:
model = Chain([
    Conv((3, 3), 3 => 128, relu, pad=SamePad(), stride=(2, 2)), #pad=SamePad() ensures size(output,d) == size(x,d) / stride.
    BatchNorm(128),
    MeanPool((2,2)),
    SkipConnection(
        Chain([
            Conv((5, 5), 128 => 128, relu, pad=SamePad(), stride=(1, 1))
            ]),
        +
    ),
    ConvTranspose((3, 3), 128 => 128, relu, pad=SamePad(), stride=(2, 2)),#pad=SamePad() ensures size(output,d) == size(x,d) * stride.
    Flux.flatten,
    Dense(32768, 100, relu),
    Dense(100, 10)
])
testmode!(model)
# image_seeds = CIFAR10(:train)[1:5].features # 32 x 32 x 3 x 5
image_seeds = [CIFAR10(:train)[i].features for i in 1:2]
# println(typeof(image_seeds[1][1,1,1,1]))
search_method = BFS(max_iter=1, batch_size=1)
split_method = Bisect(1)
output_set = BallInf(zeros(10), 1.0)

In [ ]:
prop_method = ImageStarZono()
@timed verify(search_method, split_method, prop_method, Problem(model, image_seeds, output_set))

In [ ]:
Profile.clear()
@profile verify(search_method, split_method, prop_method, Problem(model, image_seeds, output_set))

In [ ]:
open("./prof.txt", "w") do s
    Profile.print(IOContext(s, :displaysize => (24, 500)))
end